In [34]:
twin_file = '../sibpair_similarity/spark_exome_identicals.txt'
ped_file = '../data/spark_jae_adj.ped'
out_file = '../data/spark_exome.split.identicals.ped'

In [35]:
samples_of_interest = set()
with open(twin_file, 'r') as f:
    for line in f:
        individuals = [x.split('_')[-1] for x in line.strip().split('\t')]
        samples_of_interest.update(individuals)

In [36]:
family_to_children = dict()
sample_to_line = dict()
with open(ped_file, 'r') as f:
    for line in f:
        pieces = line.strip().split('\t')
        family_key = (pieces[0], pieces[2], pieces[3])
        if family_key not in family_to_children:
            family_to_children[family_key] = set()
        family_to_children[family_key].add(pieces[1])
        sample_to_line[pieces[1]] = pieces

In [37]:
with open(out_file, 'w+') as f:
    for (family_key, father_id, mother_id), children in family_to_children.items():
        identicals = children & samples_of_interest
        if len(identicals) > 0:
            # this family has identicals, create a family for each one
            nonidentical_children = children - samples_of_interest
            for identical in identicals:
                new_family_key = '%s.%s' % (family_key, identical)
                for sample_id in [father_id, mother_id, identical] + list(nonidentical_children):
                    f.write('\t'.join([new_family_key] + sample_to_line[sample_id][1:]) + '\n')